# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import os
import joblib

from azureml.train.automl import AutoMLConfig
from pprint import pprint # Used in printing automl model parameters
from azureml.core import Model # Used to get model information

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_loc = "https://raw.githubusercontent.com/khalidw/Capstone-Project-Azure-Machine-Learning-Engineer/master/divorce.csv"
ds = TabularDatasetFactory.from_delimited_files(data_loc)
data = ds.to_pandas_dataframe().dropna()

In [8]:
data.describe()

,Atr1,Atr2,Atr3,Atr4,Atr5,Atr6,Atr7,Atr8,Atr9,Atr10,...,Atr46,Atr47,Atr48,Atr49,Atr50,Atr51,Atr52,Atr53,Atr54,Class
count,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,...,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,1.776471,1.652941,1.764706,1.482353,1.541176,0.747059,0.494118,1.452941,1.458824,1.576471,...,2.552941,2.270588,2.741176,2.382353,2.429412,2.476471,2.517647,2.241176,2.011765,0.494118
std,1.627257,1.468654,1.415444,1.504327,1.632169,0.904046,0.898698,1.546371,1.557976,1.421529,...,1.371786,1.586841,1.137348,1.511587,1.405090,1.260238,1.476537,1.505634,1.667611,0.501442
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,0.000000
50%,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,2.000000,...,3.000000,2.000000,3.000000,3.000000,2.000000,3.000000,3.000000,2.000000,2.000000,0.000000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,3.000000,3.000000,3.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000


In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneExp'

experiment=Experiment(ws, experiment_name)

In [4]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
compute_name = "optCompute"
try:
    trainCluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    trainCluster = ComputeTarget.create(ws, compute_name, compute_config)
trainCluster.wait_for_completion(show_output=True)

optCompute exists already
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=trainCluster,
        training_data=ds,
        label_column_name='Class',
        n_cross_validations=5,
        **automl_settings
)

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

{'runId': 'AutoML_ce25be63-6ad2-40b3-9fb4-4e1a156aa6f9',
 'target': 'optCompute',
 'status': 'Completed',
 'startTimeUtc': '2021-01-08T18:04:19.172662Z',
 'endTimeUtc': '2021-01-08T18:37:55.162045Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'optCompute',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneExp","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-133988","workspace_name":"quick-starts-ws-133988","region":"southcentralus","compute_target":"optCompute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featur

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: CapstoneExp,
Id: AutoML_ce25be63-6ad2-40b3-9fb4-4e1a156aa6f9_40,
Type: azureml.scriptrun,
Status: Completed)
precision_score_micro 1.0
weighted_accuracy 1.0
AUC_macro 1.0
precision_score_weighted 1.0
average_precision_score_macro 1.0
accuracy 1.0
norm_macro_recall 1.0
AUC_micro 1.0
recall_score_macro 1.0
balanced_accuracy 1.0
AUC_weighted 1.0
f1_score_macro 1.0
f1_score_weighted 1.0
average_precision_score_micro 1.0
log_loss 0.03694490202784341
f1_score_micro 1.0
precision_score_macro 1.0
recall_score_weighted 1.0
matthews_correlation 1.0
average_precision_score_weighted 1.0
recall_score_micro 1.0
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_ce25be63-6ad2-40b3-9fb4-4e1a156aa6f9_40/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_ce25be63-6ad2-40b3-9fb4-4e1a156aa6f9_40/confusion_matrix


In [14]:
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

TruncatedSVDWrapper
{'n_components': 0.5047368421052632, 'random_state': None}

LightGBMClassifier
{'boosting_type': 'goss',
 'class_weight': None,
 'colsample_bytree': 0.5944444444444444,
 'importance_type': 'split',
 'learning_rate': 0.05263631578947369,
 'max_bin': 360,
 'max_depth': -1,
 'min_child_samples': 9,
 'min_child_weight': 0,
 'min_split_gain': 0.3684210526315789,
 'n_estimators': 400,
 'n_jobs': 1,
 'num_leaves': 89,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.5263157894736842,
 'reg_lambda': 0.5263157894736842,
 'silent': True,
 'subsample': 1,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'verbose': -10}



In [15]:
#TODO: Save the best model
myModel = best_run.register_model(model_path='outputs/model.pkl', model_name='capstoneModel_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(myModel)

Model(workspace=Workspace.create(name='quick-starts-ws-133988', subscription_id='d7f39349-a66b-446e-aba6-0053c2cf1c11', resource_group='aml-quickstarts-133988'), name=capstoneModel_automl, id=capstoneModel_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '1.0'})

In [16]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

capstoneModel_automl version: 1
	 Training context : Auto ML
	 Accuracy : 1.0


HyperDrive_HighAccuracy version: 1
	 Accuracy : 0.9803921568627451
	 Regularization Strength : 4.0
	 Max Iterations : 80




## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# List all files in the output
# This is used to identify the scoring script and dependencies
os.listdir('outputs')


In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


#env = Environment.get(workspace, "AzureML-Minimal").clone(env_name)

# for pip_package in ["scikit-learn"]:
#     env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='path-to-score.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

In [ ]:
import json

#Import test data
test_df = data.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('Class')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
response = service.run(input_data = test_sample)

In [ ]:
print(response)

In [ ]:
print(label_df)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()